# balanced network

Let's start with a rather complete example modified from 

http://neuralensemble.org/docs/PyNN/examples/VAbenchmarks.html


In [1]:
try:
    import pyNN.spiNNaker as sim
    simulator = 'spinnaker'
except ModuleNotFoundError:
    import pyNN.nest as sim
    simulator = 'nest'

Detected PyNN version 0.9.4 and Neo version 0.6.1



## initialization

In [2]:

# === Define parameters ========================================================

threads  = 1
rngseed  = 98765
parallel_safe = True

n        = 1000  #4000 #number of cells
r_ei     = 4.0   # number of excitatory cells:number of inhibitory cells
pconn    = 0.01  # connection probability
stim_dur = 50.   # (ms) duration of random stimulation
rate     = 10.  # (Hz) frequency of the random stimulation

dt       = 0.1   # (ms) simulation timestep
tstop    = 1000  # (ms) simulaton duration
delay    = 0.2
n_record = 40
# Cell parameters
area     = 20000. # (µm²)
tau_m    = 20.    # (ms)
cm       = 1.     # (µF/cm²)
g_leak   = 5e-5   # (S/cm²)
E_leak   = -60.  # (mV)
v_thresh = -50.   # (mV)
v_reset  = -60.   # (mV)
t_refrac = 5.     # (ms) (clamped at v_reset)
v_mean   = -60.   # (mV) 'mean' membrane potential, for calculating CUBA weights
tau_exc  = 5.     # (ms)
tau_inh  = 10.    # (ms)

# Synapse parameters
Gexc = 4.     # (nS)
Ginh = 51.    # (nS)
Erev_exc = 0.     # (mV)
Erev_inh = -80.   # (mV)

### what is the synaptic delay???

# === Calculate derived parameters =============================================

area  = area*1e-8                     # convert to cm²
cm    = cm*area*1000                  # convert to nF
Rm    = 1e-6/(g_leak*area)            # membrane resistance in MΩ
assert tau_m == cm*Rm                 # just to check
n_exc = int(round((n*r_ei/(1+r_ei)))) # number of excitatory cells
n_inh = n - n_exc                     # number of inhibitory cells
w_exc    = Gexc*1e-3              # We convert conductances to uS
w_inh    = Ginh*1e-3

# === Build the network ========================================================
extra = {'threads' : threads,
         'filename': "va.xml",
         'label': 'VA'}

cell_params = {
    'tau_m'      : tau_m,    'tau_syn_E'  : tau_exc,  'tau_syn_I'  : tau_inh,
    'v_rest'     : E_leak,   'v_reset'    : v_reset,  'v_thresh'   : v_thresh,
    'cm'         : cm,       'tau_refrac' : t_refrac}

cell_params['e_rev_E'] = Erev_exc
cell_params['e_rev_I'] = Erev_inh


## running it...


In [3]:

def run():

    from pyNN.utility import Timer, ProgressBar, init_logging, normalized_filename
    from pyNN.random import NumpyRNG, RandomDistribution
    from neo.io import NixIO

    init_logging(None, debug=False)

    timer = Timer()
    
    node_id = sim.setup(timestep=dt, min_delay=delay, max_delay=1.0, **extra)
    num_process = sim.num_processes()

    timer.start()

    print("%s Creating cell populations..." % node_id)

    # create a single population of neurons, and then use population views to define
    # excitatory and inhibitory sub-populations
    celltype = sim.IF_cond_exp
    all_cells = sim.Population(n_exc + n_inh, celltype(**cell_params), label="All Cells")
    exc_cells = all_cells[:n_exc]
    #exc_cells.label = "Excitatory cells"
    inh_cells = all_cells[n_exc:]
    #inh_cells.label = "Inhibitory cells"

    ext_stim = sim.Population(20, sim.SpikeSourcePoisson(rate=rate, duration=stim_dur), label="expoisson")
    #ext_stim = sim.SpikeSourceArray(spike_times=cellSourceSpikes)
    rconn = 0.01
    ext_conn = sim.FixedProbabilityConnector(rconn)
    ext_syn = sim.StaticSynapse(weight=0.1)

    print("%s Initialising membrane potential to random values..." % node_id)
    rng = NumpyRNG(seed=rngseed, parallel_safe=parallel_safe)
    uniformDistr = RandomDistribution('uniform', low=v_reset, high=v_thresh, rng=rng)
    all_cells.initialize(v=uniformDistr)

    print("%s Connecting populations..." % node_id)
    progress_bar = ProgressBar(width=20)
    connector = sim.FixedProbabilityConnector(pconn, rng=rng, callback=progress_bar)
    exc_syn = sim.StaticSynapse(weight=w_exc, delay=delay)
    inh_syn = sim.StaticSynapse(weight=w_inh, delay=delay)

    connections = {}
    connections['exc'] = sim.Projection(exc_cells, all_cells, connector, exc_syn, receptor_type='excitatory')
    connections['inh'] = sim.Projection(inh_cells, all_cells, connector, inh_syn, receptor_type='inhibitory')
    connections['ext'] = sim.Projection(ext_stim, all_cells, ext_conn, ext_syn, receptor_type='excitatory')

    # === Setup recording ==========================================================
    print("%s Setting up recording..." % node_id)
    all_cells.record('spikes')
    exc_cells[[0, 1]].record('v')

    buildCPUTime = timer.diff()

    # === Run simulation ===========================================================
    print("%d Running simulation..." % node_id)
    sim.run(tstop)

    E_count = exc_cells.mean_spike_count()
    I_count = inh_cells.mean_spike_count()

    #n_connections = "%d e→e,i  %d i→e,i" % (connections['exc'].size(),
    #                                      connections['inh'].size())
    simCPUTime = timer.diff()
    # === Output results to file ====================================================

    print("%d Writing data to file..." % node_id)

    #root, basename, extension = "Results", "VAbenchmarks_exc", "h5" if do_nixio else "pkl"
    root, basename = "Results", "VAbenchmarks_exc"#, "h5" if do_nixio else "pkl"

    #filename = normalized_filename(root, basename, extension, simulator)
    filenameE = "exc" #normalized_filename(root, basename)#, simulator)
    filenameI = "inh" #normalized_filename(root, basename)#, simulator)

    with NixIO(filenameE + '.h5') as io:
        exc_cells.write_data(io)
    with NixIO(filenameI + '.h5') as io:
        inh_cells.write_data(io)
    exc_cells.write_data(filenameE + '.pkl')
    inh_cells.write_data(filenameI + '.pkl')
        
    writeCPUTime = timer.diff()

    print("\n--- Vogels-Abbott Network Simulation ---")
    print("Nodes                  : %d" % 1)
    print("Number of Neurons      : %d" % n)
    #print("Number of Synapses     : %s" % n_connections)
    print("Excitatory conductance : %g nS" % Gexc)
    print("Inhibitory conductance : %g nS" % Ginh)
    print("Excitatory rate        : %g Hz" % (E_count * 1000.0 / tstop,))
    print("Inhibitory rate        : %g Hz" % (I_count * 1000.0 / tstop,))
    print("Build time             : %g s" % buildCPUTime)
    print("Simulation time        : %g s" % simCPUTime)
    print("Writing time           : %g s" % writeCPUTime)


    # === Finished with simulator ==================================================

    sim.end()

In [4]:
%load_ext watermark
%watermark -i -h -m -v -p numpy,matplotlib,pyNN,neo,nixio  -r -g -b

2020-12-04T15:13:55+00:00

CPython 3.6.9
IPython 7.15.0

numpy 1.19.0
matplotlib 3.2.2
pyNN 0.9.4
neo 0.6.1
nixio 1.4.9

compiler   : GCC 8.4.0
system     : Linux
release    : 4.4.0-87-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 32
interpreter: 64bit
host name  : 134ac8ebb3bc
Git hash   : 915b3847ee98480320aaa9827369dd0c0926b629
Git repo   : https://github.com/SpikeAI/2020-12_brainhack_Project7
Git branch : main


In [5]:
run()

2020-12-04 15:13:55 INFO: Read cfg files: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/spinnaker/.spynnaker.cfg
2020-12-04 15:13:55 INFO: Will search these locations for binaries: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/model_binaries
2020-12-04 15:13:55 WARNING: /home/spinnaker/2020-12_brainhack_Project7/output/reports has 7 old reports that have not been closed
2020-12-04 15:13:55 WARNING: /home/spinnaker/2020-12_brainhack_Project7/output/application_generated_data_files has 8 old reports that have not been closed
2020-12-04 15:13:55 WARNING: A timestep was entered that has forced sPyNNaker to automatically slow the simulation down from real time by a factor of 10. To remove this automatic behaviour, please ente

['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/spinnaker/.spynnaker.cfg']
0 Creating cell populations...
0 Initialising membrane potential to random values...
0 Connecting populations...
0 Setting up recording...
0 Running simulation...


2020-12-04 15:13:58 INFO: Time 0:00:03.314513 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2020-12-04 15:14:07 INFO: Time 0:00:08.994099 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2020-12-04 15:14:12 INFO: Time 0:00:05.242739 taken by PartitionAndPlacePartitioner
Created spalloc job 5540713
2020-12-04 15:14:12 INFO: Created spalloc job 5540713
Waiting for board power commands to complete.
2020-12-04 15:14:12 INFO: Waiting for board power commands to complete.
2020-12-04 15:14:17 INFO: Time 0:00:05.049068 taken by SpallocAllocator
2020-12-04 15:14:17 INFO: Creating transceiver for 10.11.194.17
2020-12-04 15:14:17 INFO: Working out if machine is booted
2020-12-04 15:14:21 INFO: Attempting to

0 Writing data to file...


Getting v for All Cells
|0%                          50%                         100%|
Getting spikes for All Cells
|0%                          50%                         100%|
Getting v for All Cells
|0%                          50%                         100%|
Getting spikes for All Cells
|0%                          50%                         100%|
Getting v for All Cells
|0%                          50%                         100%|
Getting spikes for All Cells
|0%                          50%                         100%|
Getting v for All Cells
|0%                          50%                         100%|


--- Vogels-Abbott Network Simulation ---
Nodes                  : 1
Number of Neurons      : 1000
Excitatory conductance : 4 nS
Inhibitory conductance : 51 nS
Excitatory rate        : 0.7675 Hz
Inhibitory rate        : 0.94 Hz
Build time             : 0.00577235 s
Simulation time        : 65.7479 s
Writing time           : 220.408 s


to import nixio and get help, run

    import nixio
    help(nixio)

on spinnaker, run 

    pip install -U neo[nixio]
    
or  

    pip install -U nixio
    
    
TODO: read and checkout what's in Results/20201202/VAbenchmarks_inh_nest_20201202-172001.h5

In [6]:
#pip list

In [7]:
ls

3_A_creating-pkl-output.ipynb      comparison.png  inh
3_B_creating-nixio-output.ipynb    dev/            inhh5
3_boilerplate.ipynb                exc             inh.h5
3_C_loading_inputs.ipynb           exch5           inh.pkl
3_D_creating-both-pkl-nixio.ipynb  exc.h5          reports/
application_generated_data_files/  exc.pkl         Results/
